# K-Cores


In this notebook, we will use cuGraph to identify the K-Cores clusters in a test graph  


_Notebook Credits_

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware     |
| --------------|------------|------------------|-----------------|--------------------|
| Brad Rees     | 10/28/2019 | created          | 0.13            | GV100, CUDA 10.2   |
| Brad Rees     | 08/16/2020 | created          | 0.15            | GV100, CUDA 10.2   |
| Don Acosta    | 07/21/2022 | updated/tested   | 22.08 nightly   | DGX Tesla V100, CUDA 11.5   |
| Ralph Liu     | 07/26/2022 | updated/tested   | 22.08 nightly   | DGX Tesla V100, CUDA 11.5   |

## Introduction

Compute the k-core of the graph G based on the out degree of its nodes. A k-core of a graph is a maximal subgraph that contains nodes of degree k or more. This call does not support a graph with self-loops and parallel edges.

For a detailed description of the algorithm see: https://en.wikipedia.org/wiki/Degeneracy_(graph_theory)


To compute the K-Core cluster in cuGraph use: <br>
* __gc = cugraph.k_core(G, k=None, core_number=None)__
    * G: A cugraph.Graph object
    * k: optional, The _k_ value to use
    * core_number: optional, Precomputed core number of the nodes of the graph G.  The see notebook on Core-Number
    
Returns:
* __gc__
    * A new graph that contains the K-Core

All vertices with the same partition ID are in the same cluster



### Some notes about vertex IDs...
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


<img src="../../img/zachary_black_lines.png" width="35%"/>


### Prep

In [3]:
# Import needed libraries
import cugraph
import cudf

# Import a built-in dataset
from cugraph.experimental.datasets import karate

### Create a Graph

In [7]:
G = karate.get_graph(fetch=True)
G = G.to_undirected()

In [8]:
print("Main Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Main Graph
	Number of Vertices: 34
	Number of Edges:    156


### Now run K-Cores

In [9]:
# Call k-cores on the graph
kcg = cugraph.k_core(G) 

RuntimeError: non-success value returned from cugraph_core_number: CUGRAPH_UNKNOWN_ERROR

In [ ]:
print("K-Core Graph")
print("\tNumber of Vertices: " + str(kcg.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg.number_of_edges()))

### Let's looks at the results
The results show that the roughly 2/3s of the edges have been removed.  
Let's look at the degrees of the vertices

In [ ]:
d = kcg.degrees()
d.sort_values(by='out_degree', ascending=False)

It is clear that only vertices with more than out_degree of 4 are connected.  
We could remove all the disconnected vertices, or do a subgraph extraction with vertices where out_degree != 0

![Karate Club](../../img/karete-kcore.jpg)

In [ ]:
# We can also just get a list of all the remaining edges as COO
coo = kcg.view_edge_list()

In [ ]:
# print out edge list
coo

### Just for fun
Let's try specifying a K value.  Looking at the original network picture, it is easy to see that most vertices have at least degree two.  
If we specify k = 2 then only one vertex should be dropped 

In [ ]:
# Call k-cores on the graph
kcg2 = cugraph.k_core(G, k=2) 

In [ ]:
print("K-Core Graph")
print("\tNumber of Vertices: " + str(kcg2.number_of_vertices()))
print("\tNumber of Edges:    " + str(kcg2.number_of_edges()))

In [ ]:
d2 = kcg2.degrees()
d2.query('out_degree == 0')

___
Copyright (c) 2019-2022, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___